In [1]:
import torch
import torch.nn.functional as F
import numpy as np
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader

# Import from your codebase
from CLAPWrapper import CLAPWrapper
from datasets.esc50 import ESC50
from datasets.tinysol import TinySOL

DATASET = ESC50

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

Device: cpu


In [2]:
print("\n" + "="*80)
print(f"Loading {DATASET.__name__} Dataset")
print("="*80)

root_path = "../data"
dataset = DATASET(root=root_path, download=False)
print(f"✅ Dataset loaded: {len(dataset)} samples")
print(f"   Classes: {len(dataset.classes)} categories")
print(f"   Sample classes: {dataset.classes[:5]}")

# Prepare text prompts
prompt = 'this is the sound of '
text_labels = [prompt + x for x in dataset.classes]
print(f"\n📝 Text prompts: {len(text_labels)} classes")


Loading ESC50 Dataset
Loading audio files


0it [00:00, ?it/s]

2000it [00:00, 10102.44it/s]

✅ Dataset loaded: 2000 samples
   Classes: 50 categories
   Sample classes: ['airplane', 'breathing', 'brushing teeth', 'can opening', 'car horn']

📝 Text prompts: 50 classes


In [10]:
# ============================================================================
# STEP 3: Initialize Models
# ============================================================================
print("\n" + "="*80)
print("Initializing Models")
print("="*80)

# Residual config con pc_weights = 1.0 (identità)
residual_config = {
    'mode': 'attention',
    'n_components_ratio': .50,
    'reweight_factor': 2.0,
    'target_layers': [0, 1, 2, 3],  # Layers dove applicare reweighting
}

print("\n🔧 Loading CLAP Standard...", end='')
clap_standard = CLAPWrapper(
    version='2023',  # or '2022'
    use_cuda=torch.cuda.is_available(),
    type='classic'
)
print("OK")

print("\n🔧 Loading ResiDualCLAP...")
clap_residual = CLAPWrapper(
    version='2023',
    use_cuda=torch.cuda.is_available(),
    type='residual',
    residual_config=residual_config
)


Initializing Models

🔧 Loading CLAP Standard...OK

🔧 Loading ResiDualCLAP...

🔧 Setup ResiDual HTSAT
Modalità: ATTENTION
Target layers: [0, 1, 2, 3]
PCA components ratio: 0.5
Reweight factor: 2.0

✓ layer_0:
  Modalità: PER-HEAD reweighting
  Num blocks: 2
  Heads per block: 4
  Total heads: 8
  Head dim: 24D → 12 PCs

✓ layer_1:
  Modalità: PER-HEAD reweighting
  Num blocks: 2
  Heads per block: 8
  Total heads: 16
  Head dim: 24D → 12 PCs

✓ layer_2:
  Modalità: PER-HEAD reweighting
  Num blocks: 6
  Heads per block: 16
  Total heads: 96
  Head dim: 24D → 12 PCs

✓ layer_3:
  Modalità: PER-HEAD reweighting
  Num blocks: 2
  Heads per block: 32
  Total heads: 64
  Head dim: 24D → 12 PCs




In [7]:
clap_standard.clap.audio_encoder.base.htsat.num_layers

4

In [8]:
clap_standard.clap.audio_encoder.base.htsat.num_features

768

In [6]:
clap_standard.clap.audio_encoder.base.htsat.num_heads

[4, 8, 16, 32]

In [4]:
PCA_SAMPLES = 200

# Prepare audio samples for PCA fitting
print(f"Collecting samples for PCA fitting ({PCA_SAMPLES} samples)...", end='')

# Create a simple dataloader wrapper per PCA fitting
class SimpleAudioDataset:
    def __init__(self, wrapper, esc50_dataset, max_samples=1000):
        self.wrapper = wrapper
        self.audio_paths = []
        for i in range(min(max_samples, len(esc50_dataset))):
            audio_path, _, _ = esc50_dataset[i]
            self.audio_paths.append(audio_path)
    
    def __len__(self):
        return len(self.audio_paths)
    
    def __getitem__(self, idx):
        audio_tensor = self.wrapper.load_audio_into_tensor(
            self.audio_paths[idx],
            self.wrapper.args.duration,
            resample=True
        )
        # ✅ Assicurati sia 1D
        if audio_tensor.dim() > 1:
            audio_tensor = audio_tensor.squeeze()
        
        return audio_tensor

# Create dataset and loader
pca_dataset = SimpleAudioDataset(clap_residual, dataset, max_samples=PCA_SAMPLES)
pca_loader = DataLoader(
    pca_dataset, 
    batch_size=25, 
    shuffle=False,
    num_workers=0,  # Start with 0 for debugging
    pin_memory=False
)
print("OK")

In [5]:
pca_dataset[0].shape

torch.Size([308700])

In [6]:
list(pca_loader)[1].shape

torch.Size([25, 308700])

In [7]:
len(dataset)

2000

In [8]:
print("\n" + "="*80)
print("Fitting PCA Components")
print("="*80)

# Fit PCA
print(f"Fitting PCA on {len(pca_dataset)} samples...")
variance_ratios = clap_residual.clap.audio_encoder.base.htsat.fit_spectral_layers(
    pca_loader,
    max_samples=PCA_SAMPLES
)

print("\n📊 PCA Variance Ratios:")
print("="*80)

# Gestione corretta della struttura gerarchica
for layer_name, layer_data in variance_ratios.items():
    print(f"\n{layer_name}:")
    
    # Controlla se è mode 'attention' (struttura annidata) o 'layer' (array diretto)
    if isinstance(layer_data, dict):
        # Mode ATTENTION: layer_data contiene blocchi
        for block_name, block_data in layer_data.items():
            print(f"  {block_name}:")
            
            # block_data contiene le teste
            for head_name, head_variance in block_data.items():
                # head_variance è l'array numpy con le variance ratios
                top3 = head_variance[:3]
                cumulative = head_variance[:3].sum()
                print(f"    {head_name}: top3={[f'{v:.3f}' for v in top3]}, cum={cumulative:.3f}")
    else:
        # Mode LAYER: layer_data è direttamente l'array di variance ratios
        top5 = layer_data[:5]
        cumulative = layer_data[:5].sum()
        print(f"  Top 5 components: {[f'{v:.4f}' for v in top5]}")
        print(f"  Cumulative variance (top 5): {cumulative:.4f}")


Fitting PCA Components
Fitting PCA on 200 samples...

📊 Fitting Spectral Layers
Modalità: ATTENTION
Target layers: [2, 3]
Max samples: 200

📦 Fase 1: Raccolta hidden states...


Raccolta: 100%|██████████| 8/8 [01:02<00:00,  7.77s/it, samples=200]



✓ Raccolti 200 samples totali

🔬 Fase 2: Calcolo PCA e inizializzazione pesi...


layer_2:

  Block 0:
    ✓ head_0: shape=torch.Size([51200, 24]), var=['0.179', '0.145', '0.119'], cum=0.444
    ✓ head_1: shape=torch.Size([51200, 24]), var=['0.206', '0.123', '0.105'], cum=0.433
    ✓ head_2: shape=torch.Size([51200, 24]), var=['0.180', '0.146', '0.108'], cum=0.434
    ✓ head_3: shape=torch.Size([51200, 24]), var=['0.199', '0.116', '0.104'], cum=0.419
    ✓ head_4: shape=torch.Size([51200, 24]), var=['0.254', '0.125', '0.118'], cum=0.497
    ✓ head_5: shape=torch.Size([51200, 24]), var=['0.307', '0.179', '0.121'], cum=0.607
    ✓ head_6: shape=torch.Size([51200, 24]), var=['0.448', '0.168', '0.094'], cum=0.711
    ✓ head_7: shape=torch.Size([51200, 24]), var=['0.422', '0.157', '0.103'], cum=0.682
    ✓ head_8: shape=torch.Size([51200, 24]), var=['0.151', '0.110', '0.091'], cum=0.352
    ✓ head_9: shape=torch.Size([51200, 24]), var=['0.168', '0.136', '0.101'], cum=0.405
    ✓ head_10: s

In [9]:
# Get text embeddings ONCE for all classes
text_embeddings = clap_standard.get_text_embeddings(text_labels)
print(f"   Text embeddings shape: {text_embeddings.shape}")

# Test on subset first (use full dataset later)
test_size = 300  # Start with 200 samples for speed
print(f"\n📊 Testing on {test_size} samples...")

y_preds_baseline, y_labels = [], []

for i in tqdm(range(test_size), desc="Baseline"):
    # Get audio file path and label
    audio_path, target, one_hot_target = dataset[-(i+1)]
    
    # Get audio embedding
    audio_embedding = clap_standard.get_audio_embeddings([audio_path], resample=True)
    
    # Compute similarity
    similarity = clap_standard.compute_similarity(audio_embedding, text_embeddings)
    
    # Get prediction
    y_pred = F.softmax(similarity.detach().cpu(), dim=1).numpy()
    y_preds_baseline.append(y_pred)
    y_labels.append(one_hot_target.detach().cpu().numpy())

# Calculate accuracy
y_labels_array = np.concatenate(y_labels, axis=0)
y_preds_baseline_array = np.concatenate(y_preds_baseline, axis=0)

baseline_acc = accuracy_score(
    np.argmax(y_labels_array, axis=1), 
    np.argmax(y_preds_baseline_array, axis=1)
)

print(f"\n✅ Baseline Accuracy: {baseline_acc:.3f} ({baseline_acc*100:.1f}%)")

   Text embeddings shape: torch.Size([50, 1024])

📊 Testing on 300 samples...


Baseline:   0%|          | 0/300 [00:00<?, ?it/s]


✅ Baseline Accuracy: 0.933 (93.3%)


In [10]:
# Get text embeddings ONCE for all classes
text_embeddings = clap_residual.get_text_embeddings(text_labels)
print(f"   Text embeddings shape: {text_embeddings.shape}")

# Test on subset first (use full dataset later)
test_size = 300  # Start with 200 samples for speed
print(f"\n📊 Testing on {test_size} samples...")

y_preds_residual, y_labels = [], []

for i in tqdm(range(test_size), desc="residual"):
    # Get audio file path and label
    audio_path, target, one_hot_target = dataset[-(i+1)]
    
    # Get audio embedding
    audio_embedding = clap_residual.get_audio_embeddings([audio_path], resample=True)
    
    # Compute similarity
    similarity = clap_residual.compute_similarity(audio_embedding, text_embeddings)
    
    # Get prediction
    y_pred = F.softmax(similarity.detach().cpu(), dim=1).numpy()
    y_preds_residual.append(y_pred)
    y_labels.append(one_hot_target.detach().cpu().numpy())

# Calculate accuracy
y_labels_array = np.concatenate(y_labels, axis=0)
y_preds_residual_array = np.concatenate(y_preds_residual, axis=0)

baseline_acc = accuracy_score(
    np.argmax(y_labels_array, axis=1), 
    np.argmax(y_preds_residual_array, axis=1)
)

print(f"\n✅ Residual Accuracy: {baseline_acc:.3f} ({baseline_acc*100:.1f}%)")

   Text embeddings shape: torch.Size([50, 1024])

📊 Testing on 300 samples...


residual:   0%|          | 0/300 [00:00<?, ?it/s]


✅ Residual Accuracy: 0.930 (93.0%)


In [11]:
audio_path, target, one_hot_target = dataset[0]

In [12]:
clap_residual.get_audio_embeddings([audio_path], resample=True)

tensor([[ 0.6079,  0.6576, -0.5101,  ...,  1.8574,  0.9557,  0.2268]])

In [13]:
clap_standard.get_audio_embeddings([audio_path], resample=True)

tensor([[ 0.7623,  0.2343, -0.5101,  ...,  1.8940,  0.9414, -0.0119]])

In [14]:
torch.allclose(x, y, atol=1e-5)

NameError: name 'x' is not defined

In [ ]:
abs(x - y).max()

tensor(1.5497e-06)

In [ ]:
abs(x - y).sum()

tensor(0.0003)

In [ ]:
len(x[0])

1024